In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os

SRC_SUBDIR = '../'
SRC_SUBDIR = os.path.abspath(SRC_SUBDIR)
if SRC_SUBDIR not in sys.path:
    print(f'Adding source directory to the sys.path: {SRC_SUBDIR!r}')
    sys.path.insert(1, SRC_SUBDIR)

Adding source directory to the sys.path: 'd:\\Work\\Job_Hunt\\Vihave.ai\\meeting-transcript-summarization'


In [2]:
%pip install -r {SRC_SUBDIR}/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [4]:
DATASET_DIR = '../data/micro_aggression'
train_df = pd.read_csv(f'{DATASET_DIR}/train.csv')

display(train_df.head())

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [5]:
train_df = train_df[['comment_text', 'target']]

display(train_df.head())

,comment_text,target
0,"This is so cool. It's like, 'would you want yo...",0.000000
1,Thank you!! This would make my life a lot less...,0.000000
2,This is such an urgent design problem; kudos t...,0.000000
3,Is this something I'll be able to install on m...,0.000000
4,haha you guys are a bunch of losers.,0.893617


In [6]:
# from huggingface_hub import snapshot_download

c:\Users\sonis\anaconda3\envs\vihave-summarizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
# snapshot_download(repo_id="google-bert/bert-base-uncased", local_dir="../models/bert-base-uncased")

Fetching 16 files:  31%|███▏      | 5/16 [00:00<00:00, 19.34it/s]


FileNotFoundError: [Errno 2] No such file or directory: '..\\models\\bert-base-uncased\\.huggingface\\download\\coreml\\fill-mask\\float32_model.mlpackage\\Data\\com.apple.CoreML\\weights\\weight.bin.c0c9f4914b4f0053785e96b8583cd59cda63c2f1b6e95a03970d772bf097e289.incomplete'

In [7]:
from transformers import AutoTokenizer

In [8]:
MODEL_DIR = '../models/bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

In [9]:
THRESHOLD = 0.5

def process_data(row, threshold = THRESHOLD):
    text = row['comment_text']
    text = str(text)
    text = " ".join(text.split())

    encodings = tokenizer(text, padding='max_length', truncation=True, max_length=128)

    label = 0
    if row['target'] >= threshold:
        label += 1
    
    encodings['labels'] = label
    encodings['text'] = text

    return encodings

In [10]:
print(process_data({
    'comment_text': 'I hate you!',
    'target': 0.9
}))

{'input_ids': [101, 1045, 5223, 2017, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
processed_data = []
# take 7000 samples from train_df
for i, row in train_df.sample(7000).iterrows():
    processed_data.append(process_data(row))

In [12]:
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, val_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

In [13]:
import pyarrow as pa
from datasets import Dataset

In [14]:
train_hg = Dataset(pa.Table.from_pandas(train_df))
val_hg = Dataset(pa.Table.from_pandas(val_df))

In [15]:
print(train_hg)

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'text', 'token_type_ids', '__index_level_0__'],
    num_rows: 5600
})


In [16]:
from transformers import AutoModelForSequenceClassification

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../models/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import TrainingArguments, Trainer

In [19]:
training_args = TrainingArguments(
    output_dir='../results',
    evaluation_strategy='epoch',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=val_hg,
    tokenizer=tokenizer
)

c:\Users\sonis\anaconda3\envs\vihave-summarizer\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer.train()

  0%|          | 0/1750 [00:00<?, ?it/s]c:\Users\sonis\anaconda3\envs\vihave-summarizer\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
                                                  
 20%|██        | 350/1750 [01:36<06:48,  3.43it/s]

{'eval_loss': 0.20304565131664276, 'eval_runtime': 9.0278, 'eval_samples_per_second': 155.076, 'eval_steps_per_second': 9.748, 'epoch': 1.0}


 29%|██▊       | 500/1750 [02:28<07:53,  2.64it/s]  

{'loss': 0.2636, 'grad_norm': 1.147784948348999, 'learning_rate': 3.571428571428572e-05, 'epoch': 1.43}


                                                  
 40%|████      | 700/1750 [04:17<10:09,  1.72it/s]

...

 86%|████████▌ | 1500/1750 [13:14<02:43,  1.53it/s]

{'loss': 0.0931, 'grad_norm': 0.14419865608215332, 'learning_rate': 7.142857142857143e-06, 'epoch': 4.29}


                                                   
100%|██████████| 1750/1750 [16:16<00:00,  1.79it/s]

{'eval_loss': 0.315032035112381, 'eval_runtime': 19.257, 'eval_samples_per_second': 72.701, 'eval_steps_per_second': 4.57, 'epoch': 5.0}
{'train_runtime': 976.8329, 'train_samples_per_second': 28.664, 'train_steps_per_second': 1.792, 'train_loss': 0.15848311397007533, 'epoch': 5.0}


TrainOutput(global_step=1750, training_loss=0.15848311397007533, metrics={'train_runtime': 976.8329, 'train_samples_per_second': 28.664, 'train_steps_per_second': 1.792, 'total_flos': 1841777387520000.0, 'train_loss': 0.15848311397007533, 'epoch': 5.0})

In [21]:
trainer.evaluate()

100%|██████████| 88/88 [00:11<00:00,  7.66it/s]


{'eval_loss': 0.315032035112381,
 'eval_runtime': 11.5238,
 'eval_samples_per_second': 121.488,
 'eval_steps_per_second': 7.636,
 'epoch': 5.0}

In [ ]:
NEW_MODEL_DIR = '../models/ToxicModel'

In [22]:
model.save_pretrained(NEW_MODEL_DIR)

In [23]:
new_model = AutoModelForSequenceClassification.from_pretrained(NEW_MODEL_DIR)

In [24]:
new_tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

In [25]:
from transformers import pipeline
from transformers import BertTokenizer, BertForSequenceClassification

In [40]:
classifier = pipeline(
    'text-classification',
    tokenizer=BertTokenizer.from_pretrained(MODEL_DIR),
    model=BertForSequenceClassification.from_pretrained(NEW_MODEL_DIR),
)

In [41]:
text = "But he was a dark guy. Good work. He was fat and Ugly fello. this is a sample review of a movie. It was an amazing work. Its worst of all."

In [42]:
sentences = text.split('. ')

In [43]:
results = classifier(sentences)

In [44]:
for sentence, results in zip(sentences, results):
    label = results['label']
    print(label)
    score = results['score']

    if label=='LABEL_1' and score > THRESHOLD:
        print(f'Microaggressive sentence detected: {sentence} (score: {score})')
    else:
        print(f'Microaggressive sentence not detected: {sentence} (score: {score})')

LABEL_0
Microaggressive sentence not detected: But he was a dark guy (score: 0.9902204275131226)
LABEL_0
Microaggressive sentence not detected: Good work (score: 0.9981346130371094)
LABEL_1
Microaggressive sentence detected: He was fat and Ugly fello (score: 0.9943736791610718)
LABEL_0
Microaggressive sentence not detected: this is a sample review of a movie (score: 0.9981743097305298)
LABEL_0
Microaggressive sentence not detected: It was an amazing work (score: 0.9982041120529175)
LABEL_0
Microaggressive sentence not detected: Its worst of all. (score: 0.9979224801063538)
